## Demo: Generating a PRF cube for extended-scene simulations

This notebook assumes you have read and run `tutorial1_corgisim.ipynb`, which walks through simulating coronagraphic observations of an on-axis host star with off-axis companions. 

Here we show how to generate a **PRF cube** (point response functions sampled over separation and azimuth) for a given Roman CGI coronagraph configuration. The PRF cube is used in the extended-scene workflow to propagate a 2D disk (or other extended emission) via field-dependent PRF convolution.

In the cells below, we import the required modules, initialize the coronagraph optics model, and compute a PRF cube that can later be saved to disk and reused.

In [ ]:
#import necessary packages
from corgisim import scene, instrument, prf_simulation, convolution
import matplotlib.pyplot as plt
import proper
import roman_preflight_proper
import astropy.io.fits as fits
roman_preflight_proper.copy_here()

### Optics setup (similar to tutorial 1)

The coronagraph/instrument configuration is unchanged from `tutorial1_corgisim.ipynb`.

Below, we configure the ROMAN CGI optical model (HLC by default)

In [ ]:
path = # Path to the directory containing the DM solutions and PRF files

bandpass = '1'
# define coronagraph properties

# --- Coronagraph / bandpass selection ---

if bandpass == '1':
    # HLC band 1F (narrow field)
    bandpass_corgisim = "1F"
    cor_type = "hlc"
    cases = ["3e-8"]
    rootname = f"hlc_ni_{cases[0]}"
    output_dim = 51
elif bandpass == '4':
    # SPC-wide band 4 (wide field)
    bandpass_corgisim = "4F"
    cor_type = "spc-wide"
    cases = ["2e-8"]
    rootname = f"spc-wide_ni_{cases[0]}"
    output_dim = 201

cgi_mode = 'excam'

# --- Load DM solutions ---
dm1 = proper.prop_fits_read( roman_preflight_proper.lib_dir + '/examples/'+rootname+'_dm1_v.fits' )
dm2 = proper.prop_fits_read( roman_preflight_proper.lib_dir + '/examples/'+rootname+'_dm2_v.fits' )

optics_keywords = {'cor_type':cor_type, 'use_errors':2, 'polaxis':10, 'output_dim':output_dim,\
                    'use_dm1':1, 'dm1_v':dm1, 'use_dm2':1, 'dm2_v':dm2,'use_fpm':1, 'use_lyot_stop':1,  'use_field_stop':1 }

# --- Optics keywords (same as Tutorial 1) ---
optics = instrument.CorgiOptics(cgi_mode, bandpass_corgisim, optics_keywords=optics_keywords, if_quiet=True)

### Define the PRF sampling grid (radius and azimuth)

To build a PRF cube, we choose a set of off-axis locations sampled on a polar grid:
- **radii** in units of λ/D
- **azimuths** in degrees

At each (r, θ) location, we will compute an off-axis coronagraphic PSF (PRF). The collection of these PRFs forms the PRF cube used later for extended-scene (2D) convolution.


In [ ]:
# --- Define the PRF sampling grid ---

# Radial sampling (in λ/D)
radii_lamD, radii_param = convolution.build_radial_grid(
    inner_step=5,   # step inside IWA (λ/D)
    mid_step=5,     # step between IWA and OWA (λ/D)
    outer_step=5,   # step beyond OWA (λ/D)
    iwa=0,          # inner working angle (λ/D)
    owa=10,         # outer working angle (λ/D)
)

# Azimuthal sampling (in degrees)
azimuths_deg, azimuth_param = convolution.build_azimuth_grid(
    step_deg=90     # azimuthal step in degrees
)

# --- Metadata: record which DM solution/coronagraph case was used ---
dm_solution = rootname

# Build a dictionary describing this PRF cube configuration
prf_dict = prf_simulation._generate_prf_dictionary(
    radii_param,
    azimuth_param,
    dm_solution
)


### PRF sampling in the focal plane

The PRF cube is sampled on a **polar grid** in the focal plane.  
Each PRF corresponds to an off-axis point source located at a specific separation and position angle:

- **Radial coordinate**: `radii_lamD` (in units of λ/D)  
- **Azimuthal coordinate**: `azimuths_deg` (in degrees)

The combination of these two arrays defines a set of sampling points  
\((r, \theta)\), which are converted to Cartesian coordinates  
\((x = r \cos\theta,\; y = r \sin\theta)\) for visualization.

Each dot in the plot below shows one location where an off-axis PRF was computed and stored in the PRF cube. During extended-scene simulations, PRFs are selected or interpolated from this grid based on the sky position of each pixel.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

# Get valid PRF sampling positions
valid_positions = convolution.get_valid_polar_positions(radii_lamD, azimuths_deg)

# Unpack to arrays
r_vals = np.array([r for r, th in valid_positions])
theta_vals = np.array([th.to_value(u.rad) for r, th in valid_positions])

# Convert to Cartesian coordinates (λ/D)
x = r_vals * np.cos(theta_vals)
y = r_vals * np.sin(theta_vals)

# Plot
plt.figure(figsize=(5, 5))
plt.scatter(x, y, s=60)

plt.axhline(0, color="gray", lw=0.5)
plt.axvline(0, color="gray", lw=0.5)

plt.gca().set_aspect("equal")
plt.xlabel("Δx (λ/D)")
plt.ylabel("Δy (λ/D)")
plt.title("Valid PRF sampling locations")

plt.show()


### Generate the PRF cube

We now compute the off-axis coronagraphic PSFs on the polar grid defined above.  
For each sampled (r, θ) location, the Roman CGI optical model is evaluated to produce a **point response function (PRF)**. These PRFs are collected into a single FITS data cube, along with metadata describing the sampling grid and the coronagraph configuration.


In [ ]:
# Generate the PRF cube
prf_hdu = prf_simulation.make_prf_cube(
    optics=optics,
    radii_lamD=radii_lamD,
    azimuths_deg=azimuths_deg,
    prf_dict=prf_dict,
    overwrite=True,
)

In [ ]:
prf_hdu.header['COMMENT'] 

### Photometric scaling note (star vs. PRF/disk)

The on-axis host-star PSF produced by `optics.get_host_star_psf(...)` is **photometrically scaled**: each wavelength slice is weighted by a target-dependent photon countrate (computed from the stellar spectrum, bandpass throughput, and collecting area) and then summed across the bandpass.

In contrast, the off-axis PRFs used for the 2D-scene convolution are generated as **band-averaged response templates** (i.e., they capture the field-dependent coronagraphic PSF shape and throughput pattern across the bandpass) and are **not** scaled to a specific target countrate by default. 

As a result, the convolved disk image should be interpreted as a **relative disk signal map**, whose absolute normalization depends on how the disk template is scaled (e.g., via the `contrast` factor and/or an optional photometric normalization step applied after convolution).

For workflows requiring a fully photometrically consistent disk+star image in photon-rate units, one can either:

- (i) generate wavelength-resolved PRFs and apply wavelength-dependent countrate weights during convolution, **or**  
- (ii) multiply the convolved disk image by the **total integrated countrate** of the disk (calculated from the disk's physical model, the telescope area, and the bandpass)

The final calibrated image in absolute physical units (e.g., photons/s) is obtained by scaling the convolution result by the total integrated flux of the disk:

$$I_{\text{final}} = I_{\text{conv}} \times F_{\text{total\_disk}}$$

Where:
* $I_{\text{conv}}$ is the output of the field-dependent convolution (using a disk template normalized to a sum of 1).
* $F_{\text{total\_disk}}$ is the total physical flux of the disk (integrated over its entire area) as seen by the telescope.

